##Import Library

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

##Load Dataset

In [ ]:
data = pd.read_csv("/content/sample_data/Dummy_data_no_user.csv")

##Select features and target

In [ ]:
features = ['age', 'height', 'weight', 'diabetes_history', 'diabetes_heritage']
target = 'recommended_sugar_intake'

print("Columns in the DataFrame:", data.columns.tolist())

Columns in the DataFrame: ['age', 'height', 'weight', 'diabetes_history', 'diabetes_heritage', 'recommended_sugar_intake']


##Handle missing values

In [ ]:
data = data.dropna()

##List of features to scale

In [ ]:
categorical_features = ['diabetes_history', 'diabetes_heritage']
numeric_features = ['age', 'height', 'weight']


##Preprocess data

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

##Linear Regression Model

In [ ]:
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with preprocessing and linear regression
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

model = Sequential([
    Dense(1, input_dim=X_train.shape[1], activation='linear')
])

##Compile the model

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

##Fit the model to the data

In [ ]:
model.fit(X_train, y_train, epochs=100, validation_split=0.2, verbose=0)

Make predictions

In [ ]:
y_pred = model.predict(X_test)

16/16 [==============================] - 0s 5ms/step


##Evalute the Model

In [ ]:
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

print(f'MSE: {mse}, RMSE: {rmse}')

MSE: 42.26395635794814, RMSE: 6.501073477353424


##Save and load the Model

In [ ]:
model.save("/content/sample_data/Recommended_Sugar_Intake_Model_TF.h5")
print("Model saved successfully")

loaded_model = tf.keras.models.load_model("/content/sample_data/Recommended_Sugar_Intake_Model_TF.h5")
print("Model loaded successfully")

Model saved successfully
Model loaded successfully


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


##Prediction on New Data

In [ ]:
def predict_sugar_intake(model, age, height, weight, diabetes_history, diabetes_heritage):
    # Create a DataFrame with the input data
    input_data = pd.DataFrame({
        'age': [age],
        'height': [height],
        'weight': [weight],
        'diabetes_history': [diabetes_history],
        'diabetes_heritage': [diabetes_heritage]
    })

    # Apply label encoders to the input data
    for feature in categorical_features:
        input_data[feature] = label_encoders[feature].transform(input_data[feature])

    # Apply the preprocessor to the input data
    input_data_processed = preprocessor.transform(input_data)

    # Predict the recommended sugar intake
    predicted_sugar_intake = model.predict(input_data_processed)

    return predicted_sugar_intake[0][0]

# Example usage of the function
example_prediction = predict_sugar_intake(loaded_model, age=44, height=175, weight=70, diabetes_history='Yes', diabetes_heritage='No')
print(f'Example Prediction: {example_prediction}')

1/1 [==============================] - 0s 107ms/step
Example Prediction: 30.46263885498047


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from tensorflow.keras.models import load_model
import tensorflow as tf

# Path to your .h5 file in Google Drive
h5_model_path = '/content/drive/MyDrive/Recommended_Sugar_Intake_Model_TF.h5'

# Load the model
model = load_model(h5_model_path)

# Define the directory for checkpoints (without the filename)
ckpt_dir = '/content/drive/MyDrive/'

# Create a tf.train.Checkpoint object
checkpoint = tf.train.Checkpoint(model=model)

# Save the checkpoint files
ckpt_path = checkpoint.save(ckpt_dir + 'model_sugar_intake')

print(f'Saved checkpoint to {ckpt_path}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved checkpoint to /content/drive/MyDrive/model_sugar_intake-1
